In [1]:
import numpy as np
import random
import os, errno
import sys
from tqdm import trange

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt

import pandas as pd
import math

#https://github.com/lkulowski/LSTM_encoder_decoder/blob/master/code/

In [2]:
torch.cuda.is_available()

True

In [3]:
data_folder = "C:\\Users\\spars\\Documents\\Master\\JHU\TML\\HomePriceBeastNew\\"

In [4]:
with open(f"{data_folder}all_model_data_sale_model.npy", 'rb') as f:
    # Format of input x: (J, batch_size, num_features(k)), y: (H, batch_size, 1)
    X_train = np.load(f, allow_pickle=True)
    y_train = np.load(f, allow_pickle=True)
    X_test = np.load(f, allow_pickle=True)
    y_test = np.load(f, allow_pickle=True)

In [5]:
with open(f"{data_folder}all_model_labels_mapping_sale_model.npy", 'rb') as f:
    train_label = np.load(f, allow_pickle=True)
    test_label = np.load(f, allow_pickle=True)

In [6]:
print(X_train.shape)
print(y_train.shape)

(5, 246249, 84)
(4, 246249, 1)


In [7]:
np.nan_to_num(X_train,copy=False)
np.nan_to_num(y_train,copy=False)
np.nan_to_num(X_test,copy=False)
np.nan_to_num(y_test,copy=False)

array([[[220600. ],
        [227850. ],
        [239100. ],
        ...,
        [179112.5],
        [188112.5],
        [182312.5]],

       [[227850. ],
        [239100. ],
        [254850. ],
        ...,
        [188112.5],
        [182312.5],
        [186062.5]],

       [[239100. ],
        [254850. ],
        [266625. ],
        ...,
        [182312.5],
        [186062.5],
        [191362.5]],

       [[254850. ],
        [266625. ],
        [253625. ],
        ...,
        [186062.5],
        [191362.5],
        [214737.5]]])

In [8]:
X_train_torch = torch.from_numpy(X_train).type(torch.Tensor).cuda()
Y_train_torch = torch.from_numpy(y_train).type(torch.Tensor).cuda()

X_test_torch = torch.from_numpy(X_test).type(torch.Tensor).cuda()
Y_test_torch = torch.from_numpy(y_test).type(torch.Tensor).cuda()

In [9]:
class inp_layer_norm(nn.Module):
    def __init__(self,input_size):
        '''
        : param input_size:     the number of features in the input X
        '''
        
        super(inp_layer_norm, self).__init__()
        
        self.layerNorm = nn.LayerNorm(input_size)
        
    def forward(self,x_input):
        n_input = self.layerNorm(x_input)
        
        return n_input

In [10]:
class lstm_final_output(nn.Module):
    '''Takes the final decoder output and the output a single feature.'''
    def __init__(self, input_size, output_size):
        '''
        : param input_size:     the number of features in the input X
        : param output_size:    the number of features in the output Y
        '''
        
        super(lstm_final_output, self).__init__()
        self.input_size = input_size
        self.output_size = output_size 
        
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x_input):
        
        '''        
        : param x_input:                    should be 2D (batch_size, input_size)
        : param encoder_hidden_states:      hidden states
        : return output, hidden:            output gives all the hidden states in the sequence;
        :                                   hidden gives the hidden state and cell state for the last
        :                                   element in the sequence 
 
        '''
        output = self.linear(x_input.squeeze(0))     
        
        return output

    
class lstm_encoder(nn.Module):
    ''' Encodes time-series sequence '''

    def __init__(self, input_size, hidden_size, dropout = 0.0, num_layers = 1):
        
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size 
        self.num_layers = num_layers
       
        # define LSTM layer
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, dropout=dropout,
                            num_layers = num_layers)
        
        
    def forward(self, x_input):
        
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''

        lstm_out, self.hidden = self.lstm(x_input.view(x_input.shape[0], x_input.shape[1], self.input_size))
        
        return lstm_out, self.hidden     
    
    def init_hidden(self, batch_size):
        
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))


class lstm_decoder(nn.Module):
    ''' Decodes hidden state output by encoder '''
    
    def __init__(self, input_size, hidden_size, dropout=0.0, num_layers = 1):

        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, dropout = dropout,
                            num_layers = num_layers)
        self.linear = nn.Linear(hidden_size, input_size)           

    def forward(self, x_input, encoder_hidden_states):
        
        '''        
        : param x_input:                    should be 2D (batch_size, input_size)
        : param encoder_hidden_states:      hidden states
        : return output, hidden:            output gives all the hidden states in the sequence;
        :                                   hidden gives the hidden state and cell state for the last
        :                                   element in the sequence 
 
        '''
        lstm_out, self.hidden = self.lstm(x_input.unsqueeze(0), encoder_hidden_states)
        output = self.linear(lstm_out.squeeze(0))     
        
        return output, self.hidden

class lstm_seq2seq(nn.Module):
    ''' train LSTM encoder-decoder and make predictions '''
    
    def __init__(self, input_size, hidden_size, output_size, dropout = 0.0, num_layers=1):

        '''
        : param input_size:     the number of expected features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param output_size:    the number of features in the output Y
        '''

        super(lstm_seq2seq, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.layer_norm = inp_layer_norm(input_size = input_size)
        self.encoder = lstm_encoder(input_size = input_size, hidden_size = hidden_size, dropout=dropout, num_layers=num_layers)
        self.decoder = lstm_decoder(input_size = input_size, hidden_size = hidden_size, dropout=dropout, num_layers=num_layers)
        self.outputter = lstm_final_output(input_size = input_size, output_size = output_size)

    def train_model(self, input_tensor, target_tensor, n_epochs, target_len, batch_size, learning_rate = 0.01):
        
        '''
        train lstm encoder-decoder
        
        : param input_tensor:              input data with shape (seq_len, # in batch, number features); PyTorch tensor    
        : param target_tensor:             target data with shape (seq_len, # in batch, number features); PyTorch tensor
        : param n_epochs:                  number of epochs 
        : param target_len:                number of values to predict 
        : param batch_size:                number of samples per gradient update
        : param learning_rate:             float >= 0; learning rate
        : return losses:                   array of loss function for each epoch
        '''
        
        # initialize array of losses 
        losses = np.full(n_epochs, np.nan)
        losses_per_horizon = np.full((n_epochs, target_len), np.nan)

        optimizer = optim.Adam(self.parameters(), lr = learning_rate)
        criterion = nn.MSELoss()

        # calculate number of batch iterations
        n_batches = int(input_tensor.shape[1] / batch_size)

        with trange(n_epochs) as tr:
            for it in tr:
                
                batch_loss = 0.
                batch_loss_tf = 0.
                batch_loss_no_tf = 0.
                num_tf = 0
                num_no_tf = 0

                for b in range(n_batches):
                    # select data 
                    input_batch = input_tensor[:, b: b + batch_size, :]
                    target_batch = target_tensor[:, b: b + batch_size, :]

                    # outputs tensor
                    outputs = torch.zeros(target_len, batch_size, target_batch.shape[2]).cuda()

                    # initialize hidden state
                    encoder_hidden = self.encoder.init_hidden(batch_size)

                    # zero the gradient
                    optimizer.zero_grad()
                    
                    #Layer normalization
                    input_batch = self.layer_norm(input_batch)

                    # encoder outputs
                    encoder_output, encoder_hidden = self.encoder(input_batch)

                    # decoder with teacher forcing
                    decoder_input = input_batch[-1, :, :]   # shape: (batch_size, input_size)
                    decoder_hidden = encoder_hidden

                    # predict recursively
                    for t in range(target_len): 
                        decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                        outputs[t] = self.outputter(decoder_output)
                        decoder_input = decoder_output

                    # compute the loss 
                    loss = criterion(outputs, target_batch)
                    loss_per_horizon = np.full(target_len, np.nan)
                    for t in range(target_len): 
                        loss_per_horizon[t] = criterion(outputs[t], target_batch[t])
                    batch_loss += loss.item()
                    
                    # backpropagation
                    loss.backward()
                    optimizer.step()

                # loss for epoch 
                batch_loss /= n_batches 
                losses[it] = batch_loss
                for t in range(target_len):
                    losses_per_horizon[it][t] = loss_per_horizon[t]
                # progress bar 
                tr.set_postfix(loss="{0:.3f}".format(batch_loss))
                    
        return losses, losses_per_horizon

    def predict(self, input_tensor, target_len):
        
        '''
        : param input_tensor:      input data (seq_len, input_size); PyTorch tensor 
        : param target_len:        number of target values to predict 
        : return np_outputs:       np.array containing predicted values; prediction done recursively 
        '''

        # encode input_tensor
        input_tensor = input_tensor.unsqueeze(1)     # add in batch size of 1
        encoder_output, encoder_hidden = self.encoder(input_tensor)

        # initialize tensor for predictions
        outputs = torch.zeros(target_len, self.output_size).cuda()

        # decode input_tensor
        decoder_input = input_tensor[-1, :, :]
        decoder_hidden = encoder_hidden
        
        for t in range(target_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = self.outputter(decoder_output)
            decoder_input = decoder_output
            
        np_outputs = outputs.cpu().detach().numpy()
        
        return np_outputs

In [11]:
def plot_train_test_results(lstm_model, Xtrain, Ytrain, Xtest, Ytest, num_rows = 4):
    '''
    plot examples of the lstm encoder-decoder evaluated on the training/test data

    : param lstm_model: trained lstm encoder-decoder
    : param Xtrain: np.array of windowed training input data
    : param Ytrain: np.array of windowed training target data
    : param Xtest: np.array of windowed test input data
    : param Ytest: np.array of windowed test target data 
    : param num_rows: number of training/test examples to plot
    : return: num_rows x 2 plots; first column is training data predictions,
    : second column is test data predictions
    '''
    offset = random.randint(5, Ytest.shape[1]-num_rows)
    # input window size
    iw = Xtrain.shape[0]
    ow = Ytest.shape[0]

    # figure setup 
    num_cols = 2
    num_plots = num_rows * num_cols

    fig, ax = plt.subplots(num_rows, num_cols, figsize = (13, 15))

    # plot training/test predictions
    for ii in range(num_rows):
        # train set
        X_train_plt = Xtrain[:, offset+ii, :]
        Y_train_pred = lstm_model.predict(torch.from_numpy(X_train_plt).type(torch.Tensor).cuda(), target_len = ow)

        ax[ii, 0].plot(np.arange(0, iw), Xtrain[:, offset+ii, 0], 'k', linewidth = 2, label = 'Input')
        ax[ii, 0].plot(np.arange(iw - 1, iw + ow), np.concatenate([[Xtrain[-1, offset+ii, 0]], Ytrain[:, offset+ii, 0]]),
                                                                 color = (0.2, 0.42, 0.72), linewidth = 2, label = 'Target')
        ax[ii, 0].plot(np.arange(iw - 1, iw + ow),        np.concatenate([[Xtrain[-1, offset+ii, 0]], Y_train_pred[:, 0]]),
                                                                 color = (0.76, 0.01, 0.01), linewidth = 2, label = 'Prediction')
        ax[ii, 0].set_xlim([0, iw + ow - 1])
        ax[ii, 0].set_xlabel('$t$')
        ax[ii, 0].set_ylabel('$y$')

        # test set
        X_test_plt = Xtest[:, offset+ii, :]
        Y_test_pred = lstm_model.predict(torch.from_numpy(X_test_plt).type(torch.Tensor).cuda(), target_len = ow)
        ax[ii, 1].plot(np.arange(0, iw), Xtest[:, offset+ii, 0], 'k', linewidth = 2, label = 'Input')
        ax[ii, 1].plot(np.arange(iw - 1, iw + ow), np.concatenate([[Xtest[-1, offset+ii, 0]], Ytest[:, offset+ii, 0]]),
                                                                 color = (0.2, 0.42, 0.72), linewidth = 2, label = 'Target')
        ax[ii, 1].plot(np.arange(iw - 1, iw + ow), np.concatenate([[Xtest[-1, offset+ii, 0]], Y_test_pred[:, 0]]),
                                                                 color = (0.76, 0.01, 0.01), linewidth = 2, label = 'Prediction')
        ax[ii, 1].set_xlim([0, iw + ow - 1])
        ax[ii, 1].set_xlabel('$t$')
        ax[ii, 1].set_ylabel('$y$')

        if ii == 0:
                ax[ii, 0].set_title('Train')

                ax[ii, 1].legend(bbox_to_anchor=(1, 1))
                ax[ii, 1].set_title('Test')

    plt.suptitle('LSTM Encoder-Decoder Predictions', x = 0.445, y = 1.)
    plt.tight_layout()
    plt.subplots_adjust(top = 0.95)
    plt.show()
#         plt.savefig('plots/predictions.png')
    plt.close() 

    return 

In [12]:
def get_prediction_result(X_test, Ytest, test_label, lstm_model):
    labels_county = test_label[0,:,0]
    labels_period = test_label[1,:,0]
    labels_state = test_label[2,:,0]
    ow = Ytest.shape[0]
    Y_test_pred = np.array([lstm_model.predict(torch.from_numpy(X_test[:,ii,:]).type(torch.Tensor).cuda(), target_len = ow) for ii in range(X_test.shape[1])])
    df = pd.DataFrame()
    df["county_name"] = labels_county
    df["period_begin"] = labels_period
    df["state_code"] = labels_state
    
    for ix in range(ow):
        df[f"y_{ix+1}"] = Ytest[ix,:,0]
        df[f"y_hat_{ix+1}"] = Y_test_pred[:,ix,0]
    
    return df

In [13]:
def plot_loss_values(loss, losses_per_horizon):
    n_epochs = len(loss)
    target_len = losses_per_horizon.shape[1]
    fig, ax = plt.subplots(2, 1, figsize = (8,8))
    col_index = ['k', 'g', 'r', 'b', 'm']
    for ii in range(target_len + 1):
        if ii == 0:
            ax[ii].plot(np.arange(1, n_epochs+1), loss, col_index[ii], linewidth = 2, label = 'Overall Loss')
            ax[ii].legend(bbox_to_anchor=(1, 1))
            ax[ii].set_title('Overall Loss')
            ax[ii].set_xlabel('Epochs')
            ax[ii].set_ylabel('Training Loss')
        else:
            ax[1].plot(np.arange(1, n_epochs+1), losses_per_horizon[:, ii-1], col_index[ii], linewidth = 1, label = f"Horizon {ii} loss")
            ax[1].legend(bbox_to_anchor=(1, 1))
            ax[1].set_title(f"Horizon loss")
            ax[1].set_xlabel('Epochs')
            ax[1].set_ylabel('Training Loss')
    plt.suptitle('Loss curves', x = 0.445, y = 1.)
    plt.tight_layout()
    plt.subplots_adjust(top = 0.95)
    plt.show()
    #   plt.savefig('plots/predictions.png')
    plt.close() 

In [14]:
# df = get_prediction_result(X_test, y_test, test_label, model)

In [15]:
# def plot_mape_dist(df):
#     for i in range(1,5):
#         df[f"mape_{i}"] = math.abs(df[f"y_hat_{i}"] - df[f"y_{i}"])/(df[f"y_{i}"] + 0.00001)

In [16]:
# plot predictions on train/test data
# plot_train_test_results(model, 
#                          X_train, 
#                          y_train, 
#                          X_test, 
#                          y_test)

# plt.close('all')

#Goals :
3) Get MAPE plots. 
1) Make the model complex. 
4) Create a grid search for params and pick best model. 
5) Repeat this for median sale price without supply. 

In [ ]:
H = 3
hidden_sizes = [10,15,30,60]
layers = [2, 4]
learning_rates = [0.02,0.05]
dropouts = [0.0]
n_epochs = 300
batch_size = 5000

best_model = None
best_loss = float("inf")
best_losses_per_horizon = None

for h in hidden_sizes:
    for l in layers: 
        for lr in learning_rates:
            for do in dropouts:
                print("-------------------------------------")
                print(f"HIDDEN SIZE {h} : NUM LAYERS : {l} : LEARNING RATE : {lr} : DROP OUT : {do}")

                model = lstm_seq2seq(input_size = X_train_torch.shape[2], hidden_size = h, dropout = do,
                                     output_size = 1, num_layers=l).cuda()
                loss, losses_per_horizon = model.train_model(X_train_torch, 
                                         Y_train_torch, 
                                         n_epochs = n_epochs, 
                                         target_len = (H+1), 
                                         batch_size = batch_size,
                                         learning_rate = lr)
                
                if loss[n_epochs - 1] < best_loss:
                    print("New best model found")
                    best_model = model
                    best_loss = loss[n_epochs - 1]
                    best_losses_per_horizon = losses_per_horizon
                    
                plot_loss_values(loss, losses_per_horizon)


                # plot predictions on train/test data
                plot_train_test_results(model, 
                                         X_train, 
                                         y_train, 
                                         X_test, 
                                         y_test)
                
                
                print("-------------------------------------")
torch.save(best_model.state_dict(), './sale_model.pth')

-------------------------------------
HIDDEN SIZE 10 : NUM LAYERS : 2 : LEARNING RATE : 0.02 : DROP OUT : 0.0


  6%|███▌                                                        | 18/300 [00:25<07:00,  1.49s/it, loss=5062625321.796]

In [ ]:
# H = 3
# h = 30
# l = 2
# lr = 0.01
# do = 0.0
# n_epochs = 300
# batch_size = 20000

# model = lstm_seq2seq(input_size = X_train_torch.shape[2], hidden_size = h, dropout = do,
#                      output_size = 1, num_layers=l).cuda()
# loss, losses_per_horizon = model.train_model(X_train_torch, 
#                          Y_train_torch, 
#                          n_epochs = n_epochs, 
#                          target_len = (H+1), 
#                          batch_size = batch_size,
#                          learning_rate = lr)
# plot_loss_values(loss, losses_per_horizon)

# torch.save(model.state_dict(), './current_sale_model.pth')

In [19]:
torch.cuda.empty_cache()

In [ ]:
plot_train_test_results(model, X_train, y_train, X_test, y_test, num_rows = 4)